<a href="https://colab.research.google.com/github/tycianojr/dataengineer/blob/main/MiniProjetoFinal_TYCIANO_JUNIOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importação da Biblioteca Pandas

In [ ]:
import pandas as pd

# Realizando a extração do dataframe

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Meus Dados/Mini Projeto Final/marketing_campaign.csv .csv",sep=",", parse_dates=['Dt_Customer'], dayfirst=True) #O tipo da coluna Dt_Customer foi alterado para parse dates
df


# Analisando o Dataframe

In [ ]:
df.head(30)

In [ ]:
#É necessario analisar as colunas com datas para converter

df.dtypes 

ID                              int64
Year_Birth                      int64
Education                      object
Marital_Status                 object
Income                        float64
Kidhome                         int64
Teenhome                        int64
Dt_Customer            datetime64[ns]
Recency                         int64
MntWines                        int64
MntFruits                       int64
MntMeatProducts                 int64
MntFishProducts                 int64
MntSweetProducts                int64
MntGoldProds                    int64
NumDealsPurchases               int64
NumWebPurchases                 int64
NumCatalogPurchases             int64
NumStorePurchases               int64
NumWebVisitsMonth               int64
AcceptedCmp3                    int64
AcceptedCmp4                    int64
AcceptedCmp5                    int64
AcceptedCmp1                    int64
AcceptedCmp2                    int64
Complain                        int64
Z_CostContac

# Tradução dos Dados para PT-BR


In [ ]:
df.rename(
    columns={
        'Year_Birth':'Ano_de_Nascimento',
       'Education':'Nivel_de_Escolaridade',
        'Marital_Status':'Estado_Civil',
        'Income':'Renda',
        'Kidhome':'Criancas_em_Casa',
        'Teenhome':'Adolescentes_em_Casa',
        'Dt_Customer':'DataDeOcorrencia',
        'Recency':'Recente',
        'MntWines':'QntVinhos',
        'MntFruits':'QntFrutas',
        'MntMeatProducts':'QntProdutos_Carne',
        'MntFishProducts':'QntProdutos_Peixe',
        'MntSweetProducts':'QntProdutos_Doce',
        'MntGoldProds':'Qnt_ProtudosDeOuro',
        'NumDealsPurchases':'NumAcordoDeCompras',
        'NumWebPurchases':'NumDeComprasViaInternet',
        'NumCatalogPurchases':'NumDeComprasViaCatalogo',
        'NumStorePurchases':'NumDeComprasNaLoja',
        'NumWebVisitsMonth':'NumDeVisitasWebMensais',
        'AcceptedCmp3':'Cmp3Recebido',
        'AcceptedCmp4':'Cmp4Recebido',
        'AcceptedCmp5':'Cmp5Recebido',
        'AcceptedCmp1':'Cmp1Recebido',
        'AcceptedCmp2':'Cmp2Recebido',
        'Complain':'Reclamacoes',
        'Response':'Resposta'
        }

, inplace=True
)

In [ ]:
#Tradução da coluna "Marital_Status"

df.loc[df.Estado_Civil == "Divorced", ['Estado_Civil']] = 'Divorciado(a)'
df.loc[df.Estado_Civil == "Together", ['Estado_Civil']] = 'Juntos'
df.loc[df.Estado_Civil == "Widow", ['Estado_Civil']] = 'Viuvo(a)'
df.loc[df.Estado_Civil == "YOLO", ['Estado_Civil']] = 'Voce vive apenas uma vez'
df.loc[df.Estado_Civil == "Married", ['Estado_Civil']] = 'Casado(a)'
df.loc[df.Estado_Civil == "Alone", ['Estado_Civil']] = 'Sozinho'
df.loc[df.Estado_Civil == "Absurd", ['Estado_Civil']] = 'Absurdo(a)'
df.loc[df.Estado_Civil == "Single", ['Estado_Civil']] = 'Solteiro(a)'

#Tradução da coluna "Education"

df.loc[df.Nivel_de_Escolaridade == "Basic", ['Nivel_de_Escolaridade']] = 'Basico'
df.loc[df.Nivel_de_Escolaridade == "Master", ['Nivel_de_Escolaridade']] = 'Mestrado'
df.loc[df.Nivel_de_Escolaridade == "2n Cycle", ['Nivel_de_Escolaridade']] = 'Especialista'
df.loc[df.Nivel_de_Escolaridade == "Graduation", ['Nivel_de_Escolaridade']] = 'Graduacao'


# Verificando inconsistência nos Dados

In [ ]:
#Verificando se os dados da coluna Z_Revenue são iguais
sorted(pd.unique(df['Z_Revenue']))

[11]

In [ ]:
#Verificando se os dados da coluna Z_CostContact são iguais
sorted(pd.unique(df['Z_CostContact'])) 

[3]

In [ ]:
sorted(pd.unique(df['Renda'])) 

In [ ]:
#Visualizando inconsistências para NaN ou NA
filter = df.Renda.isnull()
df.loc[filter]

In [ ]:
#Substitui renda por "0" para não dar drop em toda coluna e perder dados importantes
df.loc[df.Renda.isnull(), ['Renda']] = 0

# Realizar o drop(se necessário) de colunas do dataframe

In [ ]:
#Retorna valores vazios 

df.isna().sum() 

In [ ]:
#Foi observado que as colunas "Z_CostContact, Z_Revenue" possuem dados repetidos em todas as linhas. 

df.drop(['Z_CostContact','Z_Revenue'], axis=1, inplace=True)

In [ ]:
df.to_csv('PandasMarketingCampaign.csv')

# PYSPARK

In [ ]:
!pip install PySpark

In [ ]:
import pyspark.sql.functions as F
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, FloatType
from pyspark.sql.types import ArrayType, DoubleType, BooleanType
from pyspark.sql.functions import col,array_contains
from pyspark.sql.window import Window

In [ ]:
spark = (
    SparkSession.builder
      .master('local')
      .appName('dataframe_withcolumn')
      .config('spark.ui.port', '4050')
      .getOrCreate()
)

In [ ]:
#Montando a estrutura do DataFrame utilizando o StructType

Schema = StructType([
    StructField("ID", StringType(), True),
    StructField("Ano_de_Nascimento", IntegerType(), True),
    StructField("Nivel_de_Escolaridade", StringType(), True),
    StructField("Estado_Civil", StringType(), True),
    StructField("Renda", FloatType(), True),
    StructField("Criancas_em_Casa", IntegerType(), True),
    StructField("Adolescentes_em_Casa", IntegerType(), True),
    StructField("DataDeOcorrencia", StringType(), True),
    StructField("Recente", IntegerType(), True),
    StructField("QntVinhos", IntegerType(), True),
    StructField("QntFrutas", IntegerType(), True),
    StructField("QntProdutos_Carne", IntegerType(), True),
    StructField("QntProdutos_Peixe", IntegerType(), True),
    StructField("QntProdutos_Doce", IntegerType(), True),
    StructField("Qnt_ProtudosDeOuro", IntegerType(), True),
    StructField("NumAcordoDeCompras", IntegerType(), True),
    StructField("NumDeComprasViaInternet", IntegerType(), True),
    StructField("NumDeComprasViaCatalogo", IntegerType(), True),
    StructField("NumDeComprasNaLoja", IntegerType(), True),
    StructField("NumDeVisitasWebMensais", IntegerType(), True),
    StructField("Cmp3Recebido3", IntegerType(), True),
    StructField("Cmp3Recebido4", IntegerType(), True),
    StructField("Cmp3Recebido5", IntegerType(), True),
    StructField("Cmp3Recebido1", IntegerType(), True),
    StructField("Cmp3Recebido2", IntegerType(), True),
    StructField("Reclamacoes", IntegerType(), True),
    StructField("ContatoDeCusto", IntegerType(), True),
    StructField("Receita", IntegerType(), True),
    StructField("Resposta", IntegerType(), True),
])

#Lendo o arquivo CSV

local_arquivo = "/content/drive/MyDrive/Meus Dados/Mini Projeto Final/marketing_campaign.csv .csv"

df1 = spark.read.load(local_arquivo, format="csv", header="true", sep=',', schema=Schema)

In [ ]:
df1.show(30)

# Realizar a mudança de nome de pelo menos 2 colunas

In [ ]:
df1 = df1.withColumnRenamed('Renda', "Renda_Usuario")
df1 = df1.withColumnRenamed('Nivel_de_Escolaridade', "NivelDeEscolaridade")

# Deverá criar pelo menos duas novas colunas contendo alguma informação relevante sobre as outras colunas já existentes(Use a sua capacidade analítica)

In [ ]:
#Adicionando uma Coluna com o número total de alimentos

df1 = df1.withColumn("Qnt_de_Alimentos", F.col("QntVinhos") + F.col("QntFrutas") + F.col("QntProdutos_Carne") + F.col("QntProdutos_Peixe") + F.col("QntProdutos_Doce") + F.col("Qnt_ProtudosDeOuro"))

#Adicionando uma Coluna com o número total de pessoas menores de 18 anos

df1 = df1.withColumn("Menores_18_anos", F.col("Criancas_em_Casa") + F.col("Adolescentes_em_Casa"))


# Deverá utilizar filtros, ordenação e agrupamento, trazendo dados relevantes para o negócio em questão. (Use a sua capacidade analítica)

In [ ]:
#Mostra a média das Rendas de acordo com a Escolaridade

df1.groupBy(F.col("Nivel_de_Escolaridade")).avg("Renda").show()

In [ ]:
#Mostra a soma de filhos de acordo com o Estado Civil

df1.groupBy(F.col("Estado_Civil")).sum("Menores_18_anos").show()

In [ ]:
#Retorna um lista de elementos distintos (Nivel de Escolaridade)

df1.select(F.collect_set("Nivel_de_Escolaridade")).show(truncate=False)

In [ ]:
#Ordena a tabela pela quantidade de Alimentos

df1.orderBy("Qnt_de_Alimentos").show()

# Utilizar pelo menos duas Window Functions

In [ ]:
#Cria uma rank do nível de escolaridade ordenado pelo estado civil.

w0 = Window.partitionBy(F.col("Nivel_de_Escolaridade")).orderBy(F.col("Estado_Civil"))
df1.select('Nivel_de_Escolaridade', 'Estado_Civil').withColumn("dense_rank", F.dense_rank().over(w0)).show(30)


In [ ]:
#Cria um rank de Estado Civil ordenado pela Quantidade de Alimentos pulando de duas em duas linhas.

w1 = Window.partitionBy(F.col("Estado_Civil")).orderBy(F.col("Qnt_de_Alimentos").desc())

df1.select('ID', 'Estado_Civil', 'Qnt_de_Alimentos').withColumn("Próxima_Quantidade", F.lead('Qnt_de_Alimentos', 2).over(w1)).show(200)


In [ ]:
#Salva o arquivo no DRIVE

df1.write.format("csv").option("header", "true").option("inferschema", "true").option("delimiter", ",").save("/content/drive/MyDrive/SoulCodeDatas/df.marketing_campanign.csv")

# SparkSQL

In [ ]:
(spark
      .read
      .format('csv')
      .option('header', 'true')
      .option('inferschema', 'true')
      .option('delimiter', ',')
      .load('/content/drive/MyDrive/Meus Dados/Mini Projeto Final/marketing_campaign.csv .csv')
      .createOrReplaceTempView("tabela_marketing")
)

In [ ]:
spark.sql("SELECT * FROM tabela_marketing").show()


# Utilizar no minimo 10 consultas diferentes utilizando o SparkSQL, comentando o porquê de ter escolhido essas funções e explicando o que cada consulta faz.


In [ ]:
spark.sql("SELECT Education, Marital_Status, Income FROM tabela_marketing").show()

In [ ]:
#Mostram os dados referentes a Graduação e Doutorado. 
#A função WHERE filtra a coluna e o valor desejado. Importante para uma filtragem para pessoas com nível superior, neste caso.

spark.sql("SELECT * FROM tabela_marketing WHERE Education = 'Graduation' OR Education = 'PhD' OR Education = 'Master'").show()


In [ ]:
#Fazer a contagem por Estado Civil
#O COUNT é utilizado para fazer a contagem e o AS nomeia uma nova coluna.

spark.sql("SELECT Marital_Status, COUNT(Marital_Status) AS Sum_Marital FROM tabela_marketing GROUP BY Marital_Status").show()

In [ ]:
#Análise do máximo de compras de produtos (MntSweetProducts)
#O MAX filtra o maior valor.

spark.sql("SELECT MAX(MntSweetProducts) FROM tabela_marketing").show()

In [ ]:
#Análise do máximo de compras de produtos (MntSweetProducts)
#O MAX filtra o maior valor.

spark.sql("SELECT MIN(Income) FROM tabela_marketing").show()

In [ ]:
#Retorna a soma de todos os vinhos vendidos
#O comando SUM soma os valores das colunas

spark.sql("SELECT SUM(MntWines) FROM tabela_marketing").show()

In [ ]:
#Retorna a média de compras pela internet
#O comando AVG é utilizado para tirar a média e ROUND para arrendondar.

spark.sql("SELECT ROUND(AVG(NumWebPurchases), 2) AS Avg_WebPurchase FROM tabela_marketing").show()


In [ ]:
#Retorna a quantidade de vezes que o cliente retornou de acordo com seu grau de escolaridade.

spark.sql("SELECT Education, COUNT(Recency) AS Count_Recency FROM tabela_marketing WHERE Education == 'PhD' GROUP BY Education").show()

In [ ]:
#Retorna a quantidade de pessoas que tem renda maior ou igual US$100,000.00 de acordo com seu grau de escolaridade.

spark.sql("SELECT Education, COUNT(Income) AS Count_Income100k FROM tabela_marketing WHERE Income >= 100000 GROUP BY Education").show()

In [ ]:
#Utilizei o comando o LIKE e filtrei as palavras que começam com a letra "M" na coluna Marital Status

spark.sql("SELECT * FROM tabela_marketing WHERE Marital_Status LIKE 'M%'").show()

In [ ]:
#Cria uma tablema com a soma, mínimo, máximo e média de vezes que o usuário retornou de acordo com a coluna Education.

spark.sql("SELECT Condado, SUM(Casos_Confirmados) AS Sum_Casos_Confirmados, MIN(Casos_Confirmados) AS Min_Casos_Confirmados, MAX(Casos_Confirmados) AS Max_Casos_Confirmados, ROUND(AVG(Casos_Confirmados), 2) AS Avg_Casos_Confirmados FROM tabela_marketing GROUP BY Condado ORDER BY Condado").show(30)